In [1]:
!pip install pot
!pip install torch
!pip install geomloss
!pip install wget
!pip install mdatagen
!pip install pot
!pip install torch
!pip install geomloss
!pip install wget
!pip install numpy scipy sklearn
!pip install jax
!pip install chex
!pip install attr
!pip install optax



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


...

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
pip install fairlearn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd

from geomloss import SamplesLoss

import ot

import os
import pickle as pkl
import copy

from sklearn.preprocessing import scale
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer

from utils import *
from SoftImpute import softimpute, cv_softimpute
#from DataSetLoader import dataset_loader, ground_truth
from SinkhornImputation import SinkhornImputation
from Sinkhorn_CMI import SinkhornImputation_CMI
from RR_imputer import RRimputer
import matplotlib.pyplot as plt
from CMI import *
from Experiment import *

from Inject_Missing_Values import *
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from fairlearn.metrics import (
    demographic_parity_difference,  # Measures selection rate disparity
    equalized_odds_difference,     # Measures TPR/FPR disparity
      # Reports accuracy per group
)

import argparse
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test")
import pandas as pd

torch.set_default_tensor_type('torch.DoubleTensor')

c:\Users\zhossai3\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\tensor\python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)


In [2]:
import numpy as np
#from mbi import Domain, CliqueVector, Factor, LinearMeasurement
#from mbi import marginal_oracles, marginal_loss, synthetic_data
from typing import Callable
import jax
import jax.numpy as jnp
import chex
import attr
import optax

In [3]:
d =  {
    "A": [10,10,20,40,70,40,50,80,10,40,20,60,50,30,20],
    "B": [-5.23122633e-01, -3.83767348e-01,  -5.23122633e-01,  1.91159766e+00,
 -2.33927097e-01,  1.91159766e+00, -5.23122633e-01, -1.39352679e-01,
 -1.22976308e-01, -5.23122633e-01, -5.23122633e-01, 1.91159766e+00,
  1.91159766e+00, -5.23122633e-01, 1.91159766e+00 ],
    "C":[15,15,30,60,105,60,75,120,15,60,30,90,75,45,30],
    "D":[45,30,60,32,58,63,25,55,46,78,92,60,34,27,35]

}
data = pd.DataFrame(d)
data = data.to_numpy()
data_torch = torch.tensor(data)


In [4]:
data

array([[ 10.        ,  -0.52312263,  15.        ,  45.        ],
       [ 10.        ,  -0.38376735,  15.        ,  30.        ],
       [ 20.        ,  -0.52312263,  30.        ,  60.        ],
       [ 40.        ,   1.91159766,  60.        ,  32.        ],
       [ 70.        ,  -0.2339271 , 105.        ,  58.        ],
       [ 40.        ,   1.91159766,  60.        ,  63.        ],
       [ 50.        ,  -0.52312263,  75.        ,  25.        ],
       [ 80.        ,  -0.13935268, 120.        ,  55.        ],
       [ 10.        ,  -0.12297631,  15.        ,  46.        ],
       [ 40.        ,  -0.52312263,  60.        ,  78.        ],
       [ 20.        ,  -0.52312263,  30.        ,  92.        ],
       [ 60.        ,   1.91159766,  90.        ,  60.        ],
       [ 50.        ,   1.91159766,  75.        ,  34.        ],
       [ 30.        ,  -0.52312263,  45.        ,  27.        ],
       [ 20.        ,   1.91159766,  30.        ,  35.        ]])

In [5]:
bucket_space = {1:2}

In [6]:
f = bucketize(data, bucket_space)

In [9]:
print(f)

[[ 10.   0.  15.  45.]
 [ 10.   0.  15.  30.]
 [ 20.   0.  30.  60.]
 [ 40.   1.  60.  32.]
 [ 70.   0. 105.  58.]
 [ 40.   1.  60.  63.]
 [ 50.   0.  75.  25.]
 [ 80.   0. 120.  55.]
 [ 10.   0.  15.  46.]
 [ 40.   0.  60.  78.]
 [ 20.   0.  30.  92.]
 [ 60.   1.  90.  60.]
 [ 50.   1.  75.  34.]
 [ 30.   0.  45.  27.]
 [ 20.   1.  30.  35.]]


In [40]:
h =  {
    "A": [10,10,35,40,70,40,50,80,10,40,20,60,50,30,20],
    "B": [1,0,1,0,0,1,0,1,0,0,1,0,1,1,1],
    "C":[15,12,30,60,105,60,75,120,15,60,30,90,75,45,30],
    "D":[45,30,60,32,58,63,25,55,46,78,92,60,34,27,35]

}
h = pd.DataFrame(h)
h_numpy = scale(data)

In [41]:
p =h_numpy[:,1]

In [45]:
def imputation_accuracy_parity(ground_truth, imputed_data, protected_attr):
    """
    Computes the average absolute difference in imputation MSE for each feature 
    between two protected groups (0 and 1).

    Parameters:
        ground_truth (np.ndarray): Original complete dataset (n_samples, n_features)
        imputed_data (np.ndarray): Imputed dataset (n_samples, n_features)
        protected_attr (np.ndarray): Binary array (n_samples,) representing group membership

    Returns:
        float: Mean absolute MSE gap across all features
    """
    group0_mask = protected_attr == 0
    group1_mask = protected_attr == 1
    print( group0_mask)
    print( group1_mask)

    mae_diffs = []

    for i in range(ground_truth.shape[1]):
        # Get the ith feature
        gt_col = ground_truth[:, i]
        print(gt_col)
        imp_col = imputed_data[:, i]
        print(imp_col)

        # MSE for each group
        mse_0 = np.mean((gt_col[group0_mask] - imp_col[group0_mask]) )
        print(mse_0)
        mse_1 = np.mean((gt_col[group1_mask] - imp_col[group1_mask]) )
        print(mse_1)

        mae_diffs.append(abs(mse_0 - mse_1))

    return np.mean(mae_diffs)

In [46]:
iapd =imputation_accuracy_parity(data, h_numpy, p)
print(iapd)

[False False False False False False False False False False False False
 False False False]
[False False False False False False False False False False False False
 False False False]
[-1.24034735 -1.24034735 -0.77521709  0.15504342  1.55043418  0.15504342
  0.62017367  2.01556444 -1.24034735  0.15504342 -0.77521709  1.08530393
  0.62017367 -0.31008684 -0.77521709]
[-1.24034735 -1.24034735 -0.77521709  0.15504342  1.55043418  0.15504342
  0.62017367  2.01556444 -1.24034735  0.15504342 -0.77521709  1.08530393
  0.62017367 -0.31008684 -0.77521709]
nan
nan
[ 0.93541435 -1.06904497  0.93541435 -1.06904497 -1.06904497  0.93541435
 -1.06904497  0.93541435 -1.06904497 -1.06904497  0.93541435 -1.06904497
  0.93541435  0.93541435  0.93541435]
[ 0.93541435 -1.06904497  0.93541435 -1.06904497 -1.06904497  0.93541435
 -1.06904497  0.93541435 -1.06904497 -1.06904497  0.93541435 -1.06904497
  0.93541435  0.93541435  0.93541435]
nan
nan
[-1.24034735 -1.24034735 -0.77521709  0.15504342  1.55043418  

c:\Users\zhossai3\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\_core\fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\zhossai3\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\_core\_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [18]:
import torch

def bucketize_columns(data, bucket_specs):
    """
    Bucketizes specific columns in a dataset using different binning strategies.

    :param data: PyTorch tensor (N, D) (continuous values)
    :param bucket_specs: Dictionary {column_index: (method, num_bins or bin_edges)}
                         Example: {0: ('uniform', 3), 1: ('quantile', 4)}
    :return: Discretized tensor
    """
    data_buc = data.detach().clone()  # Ensure no gradients

    for col, bins in bucket_specs.items():
        feature = data_buc[:, col]  # Extract column
        #print(feature)

        
        min_val, max_val = feature.min(), feature.max()
        bin_edges = torch.linspace(min_val, max_val, bins + 1)
        print(bin_edges)
        bin_edges[-1] += 1e-6 
        
        # Apply bucketization
        data_buc[:, col] = torch.bucketize(feature, bin_edges,right=True) # Start bins from 0
        print(data_buc[:,col].long().unique())

    return data_buc.long()  # Convert to integer values

def compute_probabilities_torch(data, columns):
    """
    Compute probability distributions for given feature columns using PyTorch.

    :param data: PyTorch tensor of shape (N, D) (discretized)
    :param columns: List of column indices to compute probabilities
    :return: Unique values and probability tensor
    """
    unique_vals, counts = torch.unique(data[:, columns], dim=0, return_counts=True)
    probs = counts.float() / data.shape[0]
    return unique_vals, probs

def conditional_mutual_information_torch(data, X_cols, Y_cols, Z_cols, bucket_specs, delta=1):
    """
    Compute Conditional Mutual Information I(X;Y|Z) with bucketization inside.

    :param data: PyTorch tensor (N, D) (continuous values)
    :param X_cols: List of column indices for X
    :param Y_cols: List of column indices for Y
    :param Z_cols: List of column indices for Z
    :param bucket_specs: Dictionary specifying bucketization for each column
    :param delta: Smoothing parameter to avoid log(0)
    :return: Conditional Mutual Information (scalar)
    """
    # Apply bucketization inside CMI function
    bucketized_data = bucketize_columns(data, bucket_specs)

    # Compute probability distributions
    unique_Z, P_Z = compute_probabilities_torch(bucketized_data, Z_cols)
    unique_XZ, P_XZ = compute_probabilities_torch(bucketized_data, X_cols + Z_cols)
    unique_YZ, P_YZ = compute_probabilities_torch(bucketized_data, Y_cols + Z_cols)
    unique_XYZ, P_XYZ = compute_probabilities_torch(bucketized_data, X_cols + Y_cols + Z_cols)

    cmi = 0
    for i in range(len(unique_XYZ)):
        xyz = unique_XYZ[i]
        z = xyz[len(X_cols + Y_cols):]  # Extract Z values
        xz = torch.cat((xyz[:len(X_cols)], z))  # Concatenate X and Z
        yz = torch.cat((xyz[len(X_cols):len(X_cols + Y_cols)], z))  # Concatenate Y and Z

        mask_z = (unique_Z == z).all(dim=1)
        mask_xz = (unique_XZ == xz).all(dim=1)
        mask_yz = (unique_YZ == yz).all(dim=1)

        if mask_z.any() and mask_xz.any() and mask_yz.any():
            P_z = P_Z[mask_z].sum()
            P_xyz = P_XYZ[i]
            P_xz = P_XZ[mask_xz].sum()
            P_yz = P_YZ[mask_yz].sum()

            # Compute CMI (avoid log(0) by adding a small constant)
            cmi += delta * P_xyz * torch.log2((P_z * P_xyz) / (P_xz * P_yz + 1e-10))

    return cmi.item()


In [3]:
import torch

class CMI:
    @staticmethod
    def soft_bucketize(data, bucket_specs):
        """
        Differentiable soft binning using Gaussian kernel estimation.
        """
        data_buc = data.clone()  # Clone to prevent modifying original data

        new_data = []
        for col, bins in bucket_specs.items():
            feature = data_buc[:, col]  # Extract feature column
            min_val, max_val = feature.min(), feature.max()
            bin_centers = torch.linspace(min_val, max_val, bins, device=data_buc.device).view(-1, 1)

            # Compute soft assignments using a Gaussian kernel
            distances = torch.abs(feature.unsqueeze(1) - bin_centers.T)
            sigma = (max_val - min_val) / bins  # Bandwidth for kernel
            soft_assignments = torch.exp(-0.5 * (distances / sigma) ** 2)
            soft_assignments = soft_assignments / soft_assignments.sum(dim=1, keepdim=True)

            # Compute differentiable bin values
            new_feature = (soft_assignments @ bin_centers).squeeze()
            new_data.append(new_feature)

        return torch.stack(new_data, dim=1)  # Return as new tensor (no in-place ops)

    @staticmethod
    def soft_probability_distribution(data, columns, num_bins=10):
        """
        Compute differentiable probability estimates using soft histograms.
        """
        feature_subset = data[:, columns]
        min_val, max_val = feature_subset.min(), feature_subset.max()
        bin_centers = torch.linspace(min_val, max_val, num_bins, device=data.device).view(-1, 1)

        # Expand bin_centers to match feature dimensions
        bin_centers = bin_centers.expand(num_bins, feature_subset.shape[1])

        # Compute Gaussian kernel-based binning
        distances = torch.cdist(feature_subset, bin_centers, p=2)
        soft_counts = torch.exp(-0.5 * distances**2)
        soft_counts = soft_counts / soft_counts.sum(dim=1, keepdim=True)

        # Normalize to get probabilities
        probs = soft_counts / soft_counts.sum()
        return probs

    @staticmethod
    def conditional_mutual_information(data, X_cols, Y_cols, Z_cols, bucket_specs, delta=1):
        """
        Compute differentiable Conditional Mutual Information I(X;Y|Z).
        """
        if not data.requires_grad:
            raise ValueError("Input data must have requires_grad=True.")
        
        # Soft binning for differentiability (no in-place modification)
        bucketized_data = CMI.soft_bucketize(data, bucket_specs)

        # Compute differentiable probability distributions
        P_Z = CMI.soft_probability_distribution(bucketized_data, Z_cols)
        P_XZ = CMI.soft_probability_distribution(bucketized_data, X_cols + Z_cols)
        P_YZ = CMI.soft_probability_distribution(bucketized_data, Y_cols + Z_cols)
        P_XYZ = CMI.soft_probability_distribution(bucketized_data, X_cols + Y_cols + Z_cols)

        # Compute differentiable CMI
        cmi = torch.sum(
            P_XYZ * (
                torch.log2(P_XYZ + 1e-10) 
                + torch.log2(P_Z + 1e-10)
                - torch.log2(P_XZ + 1e-10)
                - torch.log2(P_YZ + 1e-10)
            )
        )

        return cmi





# Example usage
data = torch.randn(100, 7, requires_grad=True)  # Ensure requires_grad=True
X_cols = [0, 1]
Y_cols = [5]
Z_cols = [4]
bucket_specs = {0: 5, 1: 5, 2: 5, 3: 5, 4: 5}  # Bucket specifications

cmi_value = CMI.conditional_mutual_information(data, X_cols, Y_cols, Z_cols, bucket_specs)
print("CMI:", cmi_value)

# Compute gradients
cmi_value.backward()
print("Gradients:", data.grad)

IndexError: index 5 is out of bounds for dimension 0 with size 5

In [26]:
import torch
import numpy as np
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import scale
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

torch.set_default_tensor_type('torch.DoubleTensor')

class CMI:
    @staticmethod
    def bucketize_columns(data, bucket_specs):
        """
        Efficiently bucketizes columns using vectorized operations.
        """
        data_buc = data.clone()  # Ensure no gradients
        
        for col, bins in bucket_specs.items():
            feature = data_buc[:, col]
            min_val, max_val = feature.min(), feature.max()
            bin_edges = torch.linspace(min_val, max_val, bins + 1, device=data.device)
            bin_edges[-1] += 1e-6  # Ensure max value inclusion
            
            data_buc[:, col] = torch.searchsorted(bin_edges, feature, right=True) - 1
        
        return data_buc.long()
    
    @staticmethod
    def compute_probabilities_torch(data, columns):
        """
        Compute probability distributions efficiently.
        """
        unique_vals, counts = torch.unique(data[:, columns], dim=0, return_counts=True)
        return unique_vals, counts / data.shape[0]

    @staticmethod
    def conditional_mutual_information(data, X_cols, Y_cols, Z_cols, bucket_specs, delta=1):
        """
        Compute Conditional Mutual Information I(X;Y|Z) efficiently.
        """
        
        bucketized_data = CMI.bucketize_columns(data, bucket_specs)
            
        prob_cache = {}
        columns_list = [Z_cols, X_cols + Z_cols, Y_cols + Z_cols, X_cols + Y_cols + Z_cols]
        keys = ['Z', 'XZ', 'YZ', 'XYZ']
            
        for key, cols in zip(keys, columns_list):
            prob_cache[key] = CMI.compute_probabilities_torch(bucketized_data, cols)
            
        unique_XYZ, P_XYZ = prob_cache['XYZ']
        unique_Z, P_Z = prob_cache['Z']
        unique_XZ, P_XZ = prob_cache['XZ']
        unique_YZ, P_YZ = prob_cache['YZ']
       
            
        cmi = torch.zeros(1, device=data.device, dtype=torch.float64)  # No requires_grad=True

            
        for i, xyz in enumerate(unique_XYZ):
                z = xyz[len(X_cols + Y_cols):]
                xz = torch.cat((xyz[:len(X_cols)], z))
                yz = torch.cat((xyz[len(X_cols):len(X_cols + Y_cols)], z))
                
                mask_z = (unique_Z == z).all(dim=1)
                mask_xz = (unique_XZ == xz).all(dim=1)
                mask_yz = (unique_YZ == yz).all(dim=1)
                
                if mask_z.any() and mask_xz.any() and mask_yz.any():
                    P_z = P_Z[mask_z].sum()
                    P_xyz = P_XYZ[i]
                    P_xz = P_XZ[mask_xz].sum()
                    P_yz = P_YZ[mask_yz].sum()
                    
                    P_z.requires_grad=True
                    P_xyz.requires_grad=True
                    P_xz.requires_grad=True
                    P_yz.requires_grad=True
                    cmi_values =  P_xyz * torch.log2((P_z * P_xyz) / (P_xz * P_yz + 1e-10))
                    cmi = torch.sum(cmi_values)

    
    # Ensure non-negative CMI using torch.clamp()
        #return torch.clamp(cmi, min=0.0)
        return cmi
            
       

In [27]:
data = torch.randn(100, 7, requires_grad=True)  # Ensure requires_grad=True
X_cols = [0, 1]
Y_cols = [6]
Z_cols = [4]
bucket_specs = {0: 5, 1: 5, 3: 5, 4: 5}  # Bucket specifications

cmi_value = CMI.conditional_mutual_information(data, X_cols, Y_cols, Z_cols, bucket_specs)
print("CMI:", cmi_value)

CMI: tensor(0.0193, grad_fn=<SumBackward0>)


In [36]:
print("CMI grad_fn:", cmi_value.grad_fn)  # Should NOT be None


CMI grad_fn: None


In [35]:
import torch

def calculate_conditional_mutual_information(prob_XYZ, prob_XZ_b, prob_YZ_b, prob_Z_b):
    """
    Calculate Conditional Mutual Information (CMI) using PyTorch.

    Returns a differentiable PyTorch tensor.
    """
    # Compute CMI values
    cmi_values = prob_XYZ * torch.log2(
        (prob_XYZ * prob_Z_b) / (prob_XZ_b * prob_YZ_b + 1e-10)  # Small constant for stability
    )
    # Sum over all values
    cmi = torch.sum(cmi_values)
    
    # Ensure non-negative CMI using torch.clamp()
    return torch.clamp(cmi, min=0.0)

# --- Create Sample Differentiable Data ---
num_samples = 100
prob_XYZ = torch.randn(num_samples, requires_grad=True)  # Continuous, differentiable
prob_XZ_b = torch.randn(num_samples, requires_grad=True)
prob_YZ_b = torch.randn(num_samples, requires_grad=True)
prob_Z_b = torch.randn(num_samples, requires_grad=True)

# --- Compute CMI ---
cmi_result = calculate_conditional_mutual_information(prob_XYZ, prob_XZ_b, prob_YZ_b, prob_Z_b)

# ✅ Check `grad_fn` (it should NOT be None)
print("CMI:", cmi_result)
print("CMI grad_fn:", cmi_result.grad_fn)  # Should NOT be None (proves differentiability)

# ✅ Compute Gradients
cmi_result.backward()

# ✅ Check if gradients are computed
print("Gradient of prob_XYZ:", prob_XYZ.grad)  # Should NOT be None
print("Gradient of prob_XZ_b:", prob_XZ_b.grad)  # Should NOT be None
print("Gradient of prob_YZ_b:", prob_YZ_b.grad)  # Should NOT be None
print("Gradient of prob_Z_b:", prob_Z_b.grad)  # Should NOT be None


CMI: tensor(nan, grad_fn=<ClampBackward1>)
CMI grad_fn: <ClampBackward1 object at 0x00000228F9CB8490>
Gradient of prob_XYZ: tensor([0., nan, 0., 0., 0., 0., nan, 0., 0., nan, nan, 0., nan, 0., 0., 0., nan, nan, nan, 0., nan, 0., nan, nan,
        0., 0., nan, nan, 0., 0., 0., nan, 0., nan, nan, 0., nan, 0., 0., 0., 0., nan, nan, 0., nan, nan, nan, nan,
        0., 0., nan, nan, 0., nan, 0., nan, 0., 0., 0., nan, nan, 0., 0., 0., 0., nan, nan, nan, nan, nan, 0., 0.,
        nan, 0., nan, 0., 0., 0., 0., nan, nan, 0., 0., nan, 0., nan, nan, 0., 0., 0., 0., 0., nan, 0., nan, nan,
        nan, nan, 0., nan])
Gradient of prob_XZ_b: tensor([-0., 0., 0., 0., 0., 0., -0., -0., -0., -0., -0., -0., -0., -0., 0., -0., -0., 0., 0., -0., -0., 0., -0., 0.,
        0., 0., -0., -0., 0., 0., -0., -0., 0., -0., -0., 0., -0., 0., 0., -0., 0., 0., 0., -0., -0., 0., 0., -0.,
        0., 0., 0., 0., 0., -0., -0., -0., 0., 0., 0., 0., 0., 0., -0., 0., 0., 0., 0., -0., -0., -0., 0., 0.,
        -0., -0., -0.

In [37]:
import torch

def calculate_conditional_mutual_information(prob_XYZ, prob_XZ, prob_YZ, prob_Z):
    """
    Calculate Conditional Mutual Information (CMI) using PyTorch.

    Parameters:
    - prob_XYZ: Torch tensor for P(X, Y, Z).
    - prob_XZ: Torch tensor for P(X, Z).
    - prob_YZ: Torch tensor for P(Y, Z).
    - prob_Z: Torch tensor for P(Z).

    Returns:
    - Torch scalar: Conditional mutual information (CMI).
    """
    # Compute CMI using the same formulation as JAX
    cmi_values = prob_XYZ * torch.log2(
        (prob_XYZ * prob_Z) / (prob_XZ * prob_YZ + 1e-10)  # Small constant for stability
    )

    # Sum over all values to get the final CMI
    cmi = torch.sum(cmi_values)

    # Ensure non-negative CMI using torch.clamp()
    return torch.clamp(cmi, min=0.0)  # Equivalent to jax.lax.max(cmi, 0)

def compute_probabilities_torch(data, columns):
    """
    Compute soft probability distributions using kernel-based estimation.
    """
    feature_subset = data[:, columns]
    num_samples = feature_subset.shape[0]

    # Estimate probability by applying softmax
    probs = torch.softmax(feature_subset, dim=0)

    # Normalize to ensure valid probability distribution
    return probs / torch.sum(probs)

def conditional_mutual_information(data, X_cols, Y_cols, Z_cols):
    """
    Compute differentiable Conditional Mutual Information (CMI) in PyTorch.
    """
    # Compute probability distributions
    prob_Z = compute_probabilities_torch(data, Z_cols)
    prob_XZ = compute_probabilities_torch(data, X_cols + Z_cols)
    prob_YZ = compute_probabilities_torch(data, Y_cols + Z_cols)
    prob_XYZ = compute_probabilities_torch(data, X_cols + Y_cols + Z_cols)

    # Compute differentiable CMI
    cmi = calculate_conditional_mutual_information(prob_XYZ, prob_XZ, prob_YZ, prob_Z)

    return cmi


In [ ]:
import torch
import torch.nn as nn

class CMI:
    @staticmethod
    def soft_bucketize(data, bucket_specs):
   
        bucketized_data = data.clone()  

        for col, bins in bucket_specs.items():
            feature = bucketized_data[:, col]  # Select column
            min_val, max_val = feature.min(), feature.max()
            bin_centers = torch.linspace(min_val, max_val, bins, device=data.device)

            # Compute soft bin assignments using Gaussian kernel
            distances = torch.abs(feature.unsqueeze(1) - bin_centers.unsqueeze(0))
            sigma = (max_val - min_val) / bins  # Bandwidth
            soft_assignments = torch.exp(-0.5 * (distances / sigma) ** 2)
            soft_assignments = soft_assignments / soft_assignments.sum(dim=1, keepdim=True)

            # Compute differentiable bin values
            new_feature = (soft_assignments @ bin_centers).squeeze()

            
            bucketized_data = torch.cat([
            bucketized_data[:, :col], 
            new_feature.unsqueeze(1),  
            bucketized_data[:, col+1:]
        ], dim=1)  

        return bucketized_data 


    
 
    @staticmethod
    def compute_probabilities_torch(data, columns, num_bins=10):
        """
        Compute differentiable soft probability distributions.
        """
        feature_subset = data[:, columns]  # Extract relevant columns
        print(feature_subset)
        min_val, max_val = feature_subset.min(), feature_subset.max()
        
        
        bin_centers = torch.linspace(min_val, max_val, num_bins, device=data.device, dtype=torch.float64).unsqueeze(1).repeat(1, feature_subset.shape[1])


       
        distances = torch.abs(feature_subset.unsqueeze(1) - bin_centers.unsqueeze(0))
        soft_counts = torch.exp(-0.5 * distances**2)  # Gaussian kernel
        soft_counts = soft_counts / soft_counts.sum(dim=1, keepdim=True)  # Normalize

        # Compute soft probabilities
        probs = soft_counts / soft_counts.sum(dim=0, keepdim=True)  


        return probs


        
    
    @staticmethod
    
    def conditional_mutual_information(data, X_cols, Y_cols, Z_cols, bucket_specs):
        """
        Compute fully differentiable Conditional Mutual Information (CMI).
        """
        if not data.requires_grad:
            raise ValueError("Input data must have requires_grad=True.")

       
        bucketized_data = CMI.soft_bucketize(data, bucket_specs)

        # Compute probability distributions
        prob_cache = {}
        columns_list = [Z_cols, X_cols + Z_cols, Y_cols + Z_cols, X_cols + Y_cols + Z_cols]
        keys = ['Z', 'XZ', 'YZ', 'XYZ']

        for key, cols in zip(keys, columns_list):
            prob_cache[key] = CMI.compute_probabilities_torch(bucketized_data, cols)

        # Retrieve precomputed probabilities
        P_Z = prob_cache['Z']
        P_XZ = prob_cache['XZ']
        P_YZ = prob_cache['YZ']
        P_XYZ = prob_cache['XYZ']

        cmi_values = P_XYZ * torch.log2(
        (P_XYZ * P_Z) / (P_XZ * P_YZ + 1e-10)  # Small constant for stability
    )

    # Sum over all values to get the final CMI
        cmi = torch.sum(cmi_values)
       

        return cmi  

In [9]:
# Generate synthetic data
data = torch.randn(100, 7, dtype=torch.float64, requires_grad=True)  # Ensure requires_grad=True

# Define columns for CMI computation
X_cols = [0, 1]  # Predictor variables
Y_cols = [6]  # Target variable
Z_cols = [5,4]  # Conditional variable

# Define bucket specifications (Ensure all columns used are included)
bucket_specs = {0: 5, 1: 5, 4: 5,5:5, 6: 5}  
# Compute Conditional Mutual Information (CMI)
cmi_value = CMI.conditional_mutual_information(data, X_cols, Y_cols, Z_cols, bucket_specs)


print("CMI Estimate:", cmi_value)
#print("CMI grad_fn:", cmi_value.grad_fn)  # Should NOT be None


#cmi_value.backward()


#print("Gradient of data:", data.grad)  # Should NOT be None


AttributeError: 'list' object has no attribute 'items'

In [10]:
import pandas as pd

# Read CSV file into a Pandas DataFrame
groundTruth = pd.read_csv(r'C:\Users\zhossai3\Desktop\Fair_Imputation\Data\Diabetic_Ground_Truth.csv', delimiter=',', header=0)

# Store feature columns in a DataFrame


X= groundTruth.iloc[:, :-1]  # Selects all rows and all columns except the last one
Y = groundTruth.iloc[:, -1]  # Selects all rows and only the last column


In [11]:
groundTruth_tensor = torch.tensor(scale(groundTruth)) #converting groundTruth to Tensor, z-score scaling
groundTruth_tensor.requires_grad = True

In [13]:
bucket_specs = {
     
    0: 4,   # Column 0 → age (4 bins)
    1: 2,  # Column 1 → gender (2 bins)
    17: 2,  # Column 17 → label  (2 bins)
       # Column 2 → Family Diabetic (2 bins)
       # Column 3 → HighBP (2 bins)
    4: 4, #column 4 ->  PhysicallyActive (4 bins)
    5: 20 #column 5-> BMI (20 bins)
    
}

# Define multiple attributes for X, Y, Z
X_cols = [0,1]  # Bucketized sensitive attributes (e.g., sex, race, age)
Y_cols = [17]     # Bucketized outcome-related attributes
Z_cols = [4,5]  
#Z_cols = [2, 5, 6, 10, 13, 14]

In [34]:

# Compute Conditional Mutual Information (CMI)
cmi_value = CMI.conditional_mutual_information(groundTruth_tensor, X_cols, Y_cols, Z_cols, bucket_specs)


print("CMI Estimate:", cmi_value)
print("CMI grad_fn:", cmi_value.grad_fn)  # Should NOT be None


cmi_value.backward()


print("Gradient of data:", data.grad)  # Should NOT be None


CMI Estimate: tensor(0.2630)
CMI grad_fn: None


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [14]:
def bucketize_tensor(data, bucket_specs):
        """Bucketizes each column of the PyTorch tensor based on `bucket_specs`."""
        bucketized_data = data.clone()  # Clone to avoid modifying the original tensor

        for col, bins in bucket_specs.items():
            feature = bucketized_data[:, col]  # Select column

            # Compute quantiles for bin edges
            quantiles = torch.linspace(0, 1, bins + 1, device=data.device)
            bin_edges = torch.quantile(feature, quantiles)

            # Compute soft bin assignments using Gaussian kernel
            bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2  # Use bin centers as reference points
            distances = torch.abs(feature.unsqueeze(1) - bin_centers.unsqueeze(0))
            sigma = (bin_edges[-1] - bin_edges[0]) / bins  # Bandwidth
            soft_assignments = torch.exp(-0.5 * (distances / sigma) ** 2)
            soft_assignments = soft_assignments / soft_assignments.sum(dim=1, keepdim=True)

            # Compute differentiable bin values
            new_feature = (soft_assignments @ bin_centers).squeeze()

            # Replace the original column with the bucketized feature
            bucketized_data = torch.cat([
                bucketized_data[:, :col], 
                new_feature.unsqueeze(1),  
                bucketized_data[:, col+1:]
            ], dim=1)  

        return bucketized_data 

In [19]:

def bucketize_tensor2(data, bucket_specs):
    """Bucketizes each column of the PyTorch tensor based on `bucket_specs`."""
    bucketized_data = data.clone()  

    for col, bins in bucket_specs.items():
            feature = bucketized_data[:, col]  # Select column
            min_val, max_val = feature.min(), feature.max()
            bin_centers = torch.linspace(min_val, max_val, bins, device=data.device)

            # Compute soft bin assignments using Gaussian kernel
            distances = torch.abs(feature.unsqueeze(1) - bin_centers.unsqueeze(0))
            sigma = (max_val - min_val) / bins  # Bandwidth
            soft_assignments = torch.exp(-0.5 * (distances / sigma) ** 2)
            soft_assignments = soft_assignments / soft_assignments.sum(dim=1, keepdim=True)

            # Compute differentiable bin values
            new_feature = (soft_assignments @ bin_centers).squeeze()

            
            bucketized_data = torch.cat([
            bucketized_data[:, :col], 
            new_feature.unsqueeze(1),  
            bucketized_data[:, col+1:]
        ], dim=1)  

    return bucketized_data

In [18]:
d = bucketize_tensor(groundTruth_tensor,bucket_specs)
print(torch.unique(d[:,1]))

tensor([-0.0614,  0.3921], grad_fn=<Unique2Backward0>)


In [20]:
d = bucketize_tensor2(groundTruth_tensor,bucket_specs)
print(torch.unique(d[:,1]))

tensor([-1.0343,  0.5355], grad_fn=<Unique2Backward0>)


In [4]:
import pandas as pd

# Read CSV file into a Pandas DataFrame
groundTruth = pd.read_csv(r'C:\Users\zhossai3\Desktop\Fair_Imputation\Data\Diabetic_Ground_Truth.csv', delimiter=',', header=0)

# Store feature columns in a DataFrame


X= groundTruth.iloc[:, :-1]  # Selects all rows and all columns except the last one
Y = groundTruth.iloc[:, -1]  # Selects all rows and only the last column
groundTruth_tensor = torch.tensor(scale(groundTruth)) #converting groundTruth to Tensor, z-score scaling
groundTruth_tensor.requires_grad = True

In [ ]:
import torch

class CMI(torch.nn.Module):
    def __init__(self, bandwidth=1.0):
        super().__init__()
        self.bandwidth = bandwidth

    def _gaussian_kernel(self, x, y):
        """
        Compute Gaussian kernel density estimate between x and y
        x: (n, d)
        y: (m, d)
        returns: (n,)
        """
        n, d = x.shape
        m, _ = y.shape

        x = x.unsqueeze(1)  # (n, 1, d)
        y = y.unsqueeze(0)  # (1, m, d)

        diff = (x - y) / self.bandwidth
        exponent = -0.5 * torch.sum(diff ** 2, dim=-1)  # (n, m)
        kernel_vals = torch.exp(exponent) / ((2 * torch.pi) ** (d / 2) * self.bandwidth ** d)
        return kernel_vals.mean(dim=1)  # (n,)

    def estimate_cmi_kde(self, X, Y, Z):
        """
        X, Y, Z are tensors of shape (n_samples, d)
        All operations are differentiable
        """
        XYZ = torch.cat([X, Y, Z], dim=1)
        XZ = torch.cat([X, Z], dim=1)
        YZ = torch.cat([Y, Z], dim=1)

        p_xyz = self._gaussian_kernel(XYZ, XYZ)
        p_xz = self._gaussian_kernel(XZ, XZ)
        p_yz = self._gaussian_kernel(YZ, YZ)
        p_z = self._gaussian_kernel(Z, Z)

        # Avoid division by zero
        eps = 1e-10
        p_x_given_z = p_xz / (p_z + eps)
        p_y_given_z = p_yz / (p_z + eps)
        p_xy_given_z = p_xyz / (p_z + eps)

        ratio = p_xy_given_z / (p_x_given_z * p_y_given_z + eps)
        log_fraction = torch.log(ratio + eps)

        return torch.mean(log_fraction)

    def c_m_i(self, data, b, X_cols, Y_cols, Z_cols):
        X = data[:, X_cols]
        Y = data[:, Y_cols]
        Z = data[:, Z_cols]
        return self.estimate_cmi_kde(X, Y, Z)


In [39]:
import torch

class CMI:
    @staticmethod
    def _gaussian_kernel(x, y, bandwidth):
        """
        Compute Gaussian kernel density estimate between x and y
        x: (n, d)
        y: (m, d)
        bandwidth: float
        Returns: (n,) tensor of density estimates
        """
        n, d = x.shape
        x = x.unsqueeze(1)  # (n, 1, d)
        y = y.unsqueeze(0)  # (1, m, d)

        diff = (x - y) / bandwidth
        exponent = -0.5 * torch.sum(diff ** 2, dim=-1)  # (n, m)
        kernel_vals = torch.exp(exponent) / ((2 * torch.pi) ** (d / 2) * bandwidth ** d)
        return kernel_vals.mean(dim=1)  # (n,)

    @staticmethod
    def estimate_cmi_kde(X, Y, Z, bandwidth):
        """
        Estimate Conditional Mutual Information I(X; Y | Z) using KDE
        Parameters:
            X, Y, Z: torch tensors of shape (n_samples, d)
            bandwidth: float
        Returns:
            CMI estimate (scalar torch tensor)
        """
        XYZ = torch.cat([X, Y, Z], dim=1)
        XZ = torch.cat([X, Z], dim=1)
        YZ = torch.cat([Y, Z], dim=1)

        p_xyz = CMI._gaussian_kernel(XYZ, XYZ, bandwidth)
        p_xz  = CMI._gaussian_kernel(XZ, XZ, bandwidth)
        p_yz  = CMI._gaussian_kernel(YZ, YZ, bandwidth)
        p_z   = CMI._gaussian_kernel(Z, Z, bandwidth)

        eps = 1e-10
        p_x_given_z = p_xz / (p_z + eps)
        p_y_given_z = p_yz / (p_z + eps)
        p_xy_given_z = p_xyz / (p_z + eps)

        ratio = p_xy_given_z / (p_x_given_z * p_y_given_z + eps)
        log_fraction = torch.log(ratio + eps)

        return torch.mean(log_fraction)

    @staticmethod
    def c_m_i(data,b, X_cols, Y_cols, Z_cols, bandwidth=0.3):
        """
        Convenience method to extract columns and estimate CMI.
        Parameters:
            data: torch tensor (n_samples, n_features)
            X_cols, Y_cols, Z_cols: list of int column indices
            bandwidth: KDE bandwidth
        Returns:
            CMI estimate (scalar torch tensor)
        """
        X = data[:, X_cols]
        Y = data[:, Y_cols]
        Z = data[:, Z_cols]
        return CMI.estimate_cmi_kde(X, Y, Z, bandwidth)

    @staticmethod
    def conditional_mutual_information(data,b, X_cols, Y_cols, Z_cols, bandwidth=0.3):
        """
        Convenience method to extract columns and estimate CMI.
        Parameters:
            data: torch tensor (n_samples, n_features)
            X_cols, Y_cols, Z_cols: list of int column indices
            bandwidth: KDE bandwidth
        Returns:
            CMI estimate (scalar torch tensor)
        """
        X = data[:, X_cols]
        Y = data[:, Y_cols]
        Z = data[:, Z_cols]
        return CMI.estimate_cmi_kde(X, Y, Z, bandwidth)


In [38]:
bandwidth = 2
cmi_value = CMI.c_m_i(groundTruth_tensor, bandwidth, X_cols, Y_cols, Z_cols)
print(cmi_value)

tensor(0.3615, grad_fn=<MeanBackward0>)


In [29]:
X_cols = [0, 1]
Y_cols = [17]
Z_cols = [4, 5]

# Compute CMI
cmi_instance = CMI(bandwidth=0.2)
cmi_value = cmi_instance.c_m_i(groundTruth_tensor, X_cols, Y_cols, Z_cols)

print(f"Estimated Conditional Mutual Information: {cmi_value:.4f}")

TypeError: CMI.c_m_i() missing 1 required positional argument: 'Z_cols'

In [28]:
bandwidth = 2
cmi_value = CMI.c_m_i(groundTruth_tensor, bandwidth, X_cols, Y_cols, Z_cols)
print(cmi_value)

tensor(0.0573, grad_fn=<MeanBackward0>)
